## Stacked and percent barplots 

This notebook will contain examples of stakced and percent barplots created using Seaborn. Plots data used will be INE (Spanish National Institute) Spanish Office of National statistics demographic time series indicators such as total population and population by nationality as of 1st January and also by foreign population by country of birth.

In [1]:
# 1. Load required libraries
import pandas as pd
import os

We will use this website as a reference to  start creating our stakced barplots in Seaborn

<https://python-graph-gallery.com/stacked-and-percent-stacked-barplot/>

### 1. Build path to project folder

- Get current working directory and path to project folder

In [23]:
# Get current working directory
my_wd = os.getcwd()
print("My working directory is:",my_wd)

# 1. List files in \data folder to identify Excel file to be imported into Python 
new_wd_ML_using_python = os.path.join("/home","pablo","Documents","Pablo_zorin","VS_Python_GitHub_Pablo_source","ML-using-Python")

# 2.change existing working directory to ML-using-Python folder to access Excel file located in \data folder
os.chdir(new_wd_ML_using_python)

My working directory is: /home/pablo/Documents/Pablo_zorin/VS_Python_GitHub_Pablo_source/ML-using-Python


In [24]:
new_wd = os.getcwd()
print("Changed working directory to:",new_wd)

Changed working directory to: /home/pablo/Documents/Pablo_zorin/VS_Python_GitHub_Pablo_source/ML-using-Python


### 2. Build path to Exel file to be imported from \data sub-folder

- First we check the contents in our data folder to find Excel file to import into Python

In [26]:
# 3. Build path to Exel file to be imported from \data sub-folder
data_folder = os.path.join("/home","pablo","Documents","Pablo_zorin","VS_Python_GitHub_Pablo_source","ML-using-Python",
                           'data')
data_folder_contents = os.listdir(data_folder)
print('data folder contents:',data_folder_contents)



data folder contents: ['wine_quality.zip', 'winequality.names', 'Monthly-AE-Time-Series-January-2024.xls', 'INE Resident population country of birth Spain.xlsx', 'AE_Time_Series_Data_website.txt', 'winequality-red.csv', 'OCDE_countries_population_figures_1970_2022.csv', 'all_wine_reset.csv', '02 INE Spain CV population stocks and flows 2002 2025.xlsx', 'AE_Attendances_2010_2024.csv', 'ONS_Figure_2__Population_increase_in_mid-2023_was_driven_mostly_by_net_international_migration.xls', 'winequality-white.csv', 'INE total and foreign population figures Spain.xlsx', 'monthly-milk-production-pounds.csv', 'ONS_Figure_01_Long_term_emigration_immigration_net_migration.xlsx', 'ONS_long_term_immigration_end2024.xlsx', '01 INE resident population by nationality Spain and CV 2002 2024.xlsx']


- Then we build path to Excel file we will load in Python using Pandas 

In [27]:
# 3.1 Build  path to Excel file location
ine_population_nationality = os.path.join('data','INE total and foreign population figures Spain.xlsx')
print('ÍNE_population_nationality:',ine_population_nationality)


ÍNE_population_nationality: data/INE total and foreign population figures Spain.xlsx


### 3. Import Excel file into Python 

Get individual tab names from previous "INE total and foreign population figures Spain.xlsx" file

In [28]:
my_excel_file = pd.ExcelFile(ine_population_nationality)

my_excel_file.sheet_names  # see all sheet names

['INE_Foreign_population',
 'INE_Total_population',
 'INE_Total_foreign_population']

In [30]:
INEdata = pd.read_excel(ine_population_nationality,
                                  sheet_name = 'INE_Total_foreign_population',
                                  skiprows= 2
                                )
INEdata.head()

,Todas las edades,Total,Foreign population,% Foreign Nationals/Total population,Total YoY(N),Total YoY)(%),Foreign Nationals YoY(N),Foreign Total YoY)(%)
0,1 de enero de 2025,49077984,6852348.0,0.139622,NaN,NaN,NaN,NaN
1,1 de enero de 2024,48619695,6502282.0,0.133738,458289.0,0.009426,350066.0,0.053837
2,1 de enero de 2023,48085361,6089620.0,0.126642,534334.0,0.011112,412662.0,0.067765
3,1 de enero de 2022,47486727,5509046.0,0.116012,598634.0,0.012606,580574.0,0.105386
4,1 de enero de 2021,47400798,5402702.0,0.113979,85929.0,0.001813,106344.0,0.019683


- Keep only first 20 rows to skip footnotes

In [ ]:
INE_data_clean = INEdata.head(20)


In [ ]:
INE_data_clean

In [32]:
# Rename columns
INE_data_clean.columns = ['Date','Total_population','Foreign_population','Percent_foreign_population',
'Total population YoY(N)','Total population  YoY(%)','Foreign population YoY(N)','Foreign population  YoY(%)']

- Start by subsetting previous data set keeping initial variables (Date,Total_population,Foreign_population), used to derive the rest of columns in INE_data dataframe

In [33]:
INE_total_foreign_population = INE_data_clean[['Date','Total_population','Foreign_population']]
INE_total_foreign_population.head()

,Date,Total_population,Foreign_population
0,1 de enero de 2025,49077984,6852348.0
1,1 de enero de 2024,48619695,6502282.0
2,1 de enero de 2023,48085361,6089620.0
3,1 de enero de 2022,47486727,5509046.0
4,1 de enero de 2021,47400798,5402702.0


### 4. Create new calculated fields 

In [35]:
INE_total_foreign_population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Date                20 non-null     object 
 1   Total_population    20 non-null     object 
 2   Foreign_population  20 non-null     float64
dtypes: float64(1), object(2)
memory usage: 612.0+ bytes


#### 4.1 New column for year

In [34]:
INE_total_foreign_population['Year'] = INE_total_foreign_population['Date'].str_strip().str[13:]

AttributeError: 'Series' object has no attribute 'str_strip'

#### 4.1 New column for Spanish nationals from total and foreign population figures

We will compute Spanish nationals column substracting Foreign population to Total population obtaining total population split by spanish/foreign nationality

First we duplicate our original dataframe using .copy() function 

In [21]:
INE_population_nationality = INE_total_foreign_population.copy()
INE_population_nationality.head()

,Date,Total_population,Foreign_population
0,1 de enero de 2025,49077984,6852348.0
1,1 de enero de 2024,48619695,6502282.0
2,1 de enero de 2023,48085361,6089620.0
3,1 de enero de 2022,47486727,5509046.0
4,1 de enero de 2021,47400798,5402702.0


AttributeError: 'Series' object has no attribute 'str_strip'

Then we include our new Spanish nationals calculation from substracting Foreign_population to Total_population

In [13]:
INE_population_nationality['Spanish_nationals'] = INE_population_nationality['Total_population']- INE_population_nationality['Foreign_population']
INE_population_nationality.head()

,Date,Total_population,Foreign_population,Spanish_nationals
0,1 de enero de 2025,49077984,6852348.0,42225636.0
1,1 de enero de 2024,48619695,6502282.0,42117413.0
2,1 de enero de 2023,48085361,6089620.0,41995741.0
3,1 de enero de 2022,47486727,5509046.0,41977681.0
4,1 de enero de 2021,47400798,5402702.0,41998096.0


We also rename previous dataframe so new one is called "INE_spain_population" as it is just Spanish population split by nationality (Spanish nationals, foreign nationals)

In [14]:
INE_spain_population = INE_population_nationality.copy()
INE_spain_population.head()

,Date,Total_population,Foreign_population,Spanish_nationals
0,1 de enero de 2025,49077984,6852348.0,42225636.0
1,1 de enero de 2024,48619695,6502282.0,42117413.0
2,1 de enero de 2023,48085361,6089620.0,41995741.0
3,1 de enero de 2022,47486727,5509046.0,41977681.0
4,1 de enero de 2021,47400798,5402702.0,41998096.0


#### 4.2 New column to obtain Date as YYYY four digit year column 

Using str_strip().str[13:] function we extract from Date character column just the last four digits to create a new Year date column 

In [16]:
INE_spain_population['Year'] = INE_spain_population['Date'].str_strip().str[13:]

AttributeError: 'Series' object has no attribute 'str_strip'

In [ ]:
INE_spain_population.head()

### 5. Reshape data for stacked percent barplot 

We need to reshape previous dataframe so we have Spanish national and Foreign population under the same column